Test the performance of pre-trained GCNN models

In [ ]:
# Loading modules

from __future__ import print_function, division

import numpy as np
import multiprocessing

from ase import io
from ase.db import connect

from tinnet.feature.voronoi import Voronoi
from tinnet.regression.regression import Regression

In [ ]:
# Setting variables

# Optimized hyperparameters
lr = 0.0026635818126661374
atom_fea_len = 202
n_conv = 10
h_fea_len = 93
n_h = 1

descriptor = Voronoi(max_num_nbr=12,
                     radius=8,
                     dmin=0,
                     step=0.2,
                     dict_atom_fea=None)

db = connect('../Database.db')

images = np.array([r.toatoms() for r in db.select()])

d_cen = np.array([r['data']['d_cen'] for r in db.select()], dtype=np.float32)
full_width = np.array([r['data']['full_width'] for r in db.select()], dtype=np.float32)

idx = np.arange(len(images))

idx_1 = idx[:-38]
idx_2 = idx[-38:] # Last 38 images are pure metals

num = int(len(idx_1)*1.00) # % of database for training (1.00 means 100%)
np.random.seed(12345)
np.random.shuffle(idx_1)

idx_1 = idx_1[0:num]

idx = np.sort(np.concatenate((idx_1,idx_2)))

np.savetxt('index.txt', idx)

images = [images[i] for i in idx]

d_cen = np.array([d_cen[i] for i in idx])
full_width = np.array([full_width[i] for i in idx])

features = multiprocessing.Pool().map(descriptor.feas, images)

check_ans_train_mae = np.zeros((10,10))
check_ans_train_mse = np.zeros((10,10))
check_ans_val_mae = np.zeros((10,10))
check_ans_val_mse = np.zeros((10,10))
check_ans_test_mae = np.zeros((10,10))
check_ans_test_mse = np.zeros((10,10))

In [ ]:
# Test the performance of pre-trained GCNN models

for idx_validation in range(8,9):
    for idx_test in range(9,10):
        try:
            model = Regression(features,
                               d_cen,
                               phys_model='gcnn_multitask',
                               optim_algorithm='AdamW',
                               weight_decay=0.0001,
                               idx_validation=idx_validation,
                               idx_test=idx_test,
                               lr=lr,
                               atom_fea_len=atom_fea_len,
                               n_conv=n_conv,
                               h_fea_len=h_fea_len,
                               n_h=n_h,
                               full_width=full_width,
                               batch_size=64)
            
            check_ans_train_mae[idx_test,idx_validation],\
            check_ans_train_mse[idx_test,idx_validation],\
            check_ans_val_mae[idx_test,idx_validation],\
            check_ans_val_mse[idx_test,idx_validation],\
            check_ans_test_mae[idx_test,idx_validation],\
            check_ans_test_mse[idx_test,idx_validation] = model.check_loss()
        except:
            pass

np.savetxt('check_ans_train_mae.txt', check_ans_train_mae)
np.savetxt('check_ans_train_mse.txt', check_ans_train_mse)
np.savetxt('check_ans_val_mae.txt', check_ans_val_mae)
np.savetxt('check_ans_val_mse.txt', check_ans_val_mse)
np.savetxt('check_ans_test_mae.txt', check_ans_test_mae)
np.savetxt('check_ans_test_mse.txt', check_ans_test_mse)